# Проект: "SQL"

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# устанавливаем параметры
db_config = {'user': '###', # имя пользователя
'pwd': '###', # пароль
'host': '###',
'port': 6432, # порт подключения
'db': '###'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследуем таблицы

<b>Для простоты работы создам функцию для отправки и вывода запросов</b>

In [5]:
def sql_send(query):
    return pd.io.sql.read_sql(query, con = engine)

<b>Функция для проверки типов данных и пропусков и общей информации</b>

In [6]:
def sql_check(table):
    display(sql_send("SELECT * FROM {} LIMIT 5;".format(table)))
    query = """SELECT 
                    column_name, data_type
                FROM
                    INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_NAME = N'%s';""" % table
    temp = sql_send(query)
    display("Типы данных:", temp)
    total = sql_send("SELECT COUNT(*) FROM {};".format(table))
    display("Общее число данных в таблице:", total['count'][0])
    for i in temp['column_name']:
        query1 = "SELECT COUNT(*) FROM %s WHERE %s IS NULL;" % (table, i)
        display("Количество пропусков в столбце {}:".format(i), sql_send(query1))

<b>Исследуем каждую таблицу:</b>

In [7]:
sql_check('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


'Типы данных:'

,column_name,data_type
0,book_id,integer
1,author_id,integer
2,title,text
3,num_pages,integer
4,publication_date,date
5,publisher_id,integer


'Общее число данных в таблице:'

1000

...

,count
0,0


'Количество пропусков в столбце publication_date:'

,count
0,0


'Количество пропусков в столбце publisher_id:'

,count
0,0


In [8]:
sql_check('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


'Типы данных:'

,column_name,data_type
0,author_id,integer
1,author,text


'Общее число данных в таблице:'

636

'Количество пропусков в столбце author_id:'

,count
0,0


'Количество пропусков в столбце author:'

,count
0,0


In [9]:
sql_check('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


'Типы данных:'

,column_name,data_type
0,publisher_id,integer
1,publisher,text


'Общее число данных в таблице:'

340

'Количество пропусков в столбце publisher_id:'

,count
0,0


'Количество пропусков в столбце publisher:'

,count
0,0


In [10]:
sql_check('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


'Типы данных:'

,column_name,data_type
0,rating_id,integer
1,book_id,integer
2,username,text
3,rating,integer


'Общее число данных в таблице:'

6456

...

,count
0,0


'Количество пропусков в столбце username:'

,count
0,0


'Количество пропусков в столбце rating:'

,count
0,0


In [11]:
sql_check('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


'Типы данных:'

,column_name,data_type
0,review_id,integer
1,book_id,integer
2,username,text
3,text,text


'Общее число данных в таблице:'

2793

...

,count
0,0


'Количество пропусков в столбце username:'

,count
0,0


'Количество пропусков в столбце text:'

,count
0,0


### Количество книг после 1 января 2000 года

In [12]:
task1 = """
SELECT
    COUNT(DISTINCT book_id) 
FROM 
    books 
WHERE
    publication_date >= '2000-01-01';"""

In [13]:
task1_output = sql_send(task1)

In [14]:
print("Количество книг, которое было опубликовано после 1 января 2000 года: ", task1_output['count'][0])

Количество книг, которое было опубликовано после 1 января 2000 года:  821


<b>Выводы:</b>

В выборке после 1 января 2000 года вышло 821 книг из 1000 в выборке

In [15]:
task1 = """
SELECT
    COUNT(DISTINCT book_id) 
FROM 
    books 
WHERE
    publication_date >= '2000-01-01';"""

In [16]:
task1_output = sql_send(task1)

In [17]:
print("Количество книг, которое было опубликовано после 1 января 2000 года: ", task1_output['count'][0])

Количество книг, которое было опубликовано после 1 января 2000 года:  821


### Количество обзоров и средняя оценка для каждой книги

In [55]:
task2 = """
SELECT
    books.title AS title,
    SUBQ1.rev_cnt AS rev_cnt,
    SUBQ.avg_rate AS avg_rate    
FROM
    books
LEFT JOIN
    (SELECT
        book_id,
        AVG(rating) AS avg_rate
    FROM
        ratings
    GROUP BY
        book_id) AS SUBQ ON SUBQ.book_id = books.book_id
LEFT JOIN
    (SELECT
         book_id,
         COUNT(review_id) AS rev_cnt
     FROM
         reviews
     GROUP BY
         book_id) AS SUBQ1 ON SUBQ1.book_id = books.book_id
WHERE
    rev_cnt > 5 AND avg_rate > 4
ORDER BY
    avg_rate DESC;
"""

In [56]:
task2_output = sql_send(task2)

In [57]:
print("Количество обзоров и средняя оценка книг(топ5): ", task2_output)

Количество обзоров и средняя оценка книг(топ5):                                                 title  rev_cnt  avg_rate
0  Harry Potter and the Prisoner of Azkaban (Harr...        6  4.414634
1  Harry Potter and the Chamber of Secrets (Harry...        6  4.287500
2                                     The Book Thief        6  4.264151
3                                   The Glass Castle        6  4.206897
4                The Hobbit  or There and Back Again        6  4.125000
5                          Outlander (Outlander  #1)        6  4.125000
6  The Curious Incident of the Dog in the Night-Time        6  4.081081
7  The Lightning Thief (Percy Jackson and the Oly...        6  4.080645


<b>Вывод:</b>
* Книги J.K. Rowling пользуются огромной популярностью и имеют большой рейтинг 4.41, при количестве обзоров в 6

In [21]:
task2 = """
SELECT
    books.title AS title,
    SUBQ1.rev_cnt AS rev_cnt,
    SUBQ.avg_rate AS avg_rate    
FROM
    books
LEFT JOIN
    (SELECT
        book_id,
        AVG(rating) AS avg_rate
    FROM
        ratings
    GROUP BY
        book_id) AS SUBQ ON SUBQ.book_id = books.book_id
LEFT JOIN
    (SELECT
         book_id,
         COUNT(review_id) AS rev_cnt
     FROM
         reviews
     GROUP BY
         book_id) AS SUBQ1 ON SUBQ1.book_id = books.book_id

"""

In [22]:
sql_send(task2).sort_values(by="title")

,title,rev_cnt,avg_rate
853,'Salem's Lot,2.0,3.666667
78,1 000 Places to See Before You Die,1.0,2.500000
171,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
533,1491: New Revelations of the Americas Before C...,2.0,4.500000
303,1776,4.0,4.000000
...,...,...,...
606,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.666667
920,Xenocide (Ender's Saga #3),3.0,3.400000
760,Year of Wonders,4.0,3.200000
764,You Suck (A Love Story #2),2.0,4.500000


### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [23]:
task3 = """
SELECT 
    publisher, COUNT(book_id) AS book_cnt 
FROM 
    books INNER JOIN publishers ON publishers.publisher_id = books.publisher_id 
WHERE 
    num_pages > 50 
GROUP BY 
    publisher 
ORDER BY 
    book_cnt DESC LIMIT 10;"""

In [24]:
task3_output = sql_send(task3)

In [25]:
print("Топ 10 изадельств, выпустивших наибольшее число книг, больше 50 страниц: \n\n", task3_output)

Топ 10 изадельств, выпустивших наибольшее число книг, больше 50 страниц: 

                    publisher  book_cnt
0              Penguin Books        42
1                    Vintage        31
2   Grand Central Publishing        25
3           Penguin Classics        24
4           Ballantine Books        19
5                     Bantam        19
6                    Berkley        17
7         St. Martin's Press        14
8              Berkley Books        14
9  William Morrow Paperbacks        13


<b>Выводы:</b>
* Лидером из издательств является Penguin Books - 42 книги
* Второе место занимает Vintage - 31 книга
* Grand Central Publishing - 25 книг

### Автор с самой высокой средней оценкой книг

In [26]:
task4 = """
SELECT
    authors.author,
    AVG(SUBQ1.avg_rate) AS avg_rate
FROM
    books
    INNER JOIN authors ON authors.author_id = books.author_id
    RIGHT JOIN
(SELECT 
    SUBQ.book_id AS book_id,
    AVG(SUBQ.avg_rate) AS avg_rate
FROM
    (SELECT 
        book_id, 
        COUNT(DISTINCT rating_id) AS count_rate,
        AVG(rating) AS avg_rate
    FROM
        ratings
    GROUP BY
        book_id
    HAVING
        COUNT(rating) > 50) AS SUBQ
GROUP BY
    book_id) AS SUBQ1 ON SUBQ1.book_id = books.book_id
GROUP BY
    authors.author
ORDER BY
    avg_rate DESC;
"""

In [27]:
task4_output = sql_send(task4)

In [28]:
print("Автор с самой высокой средней оценкой книг: \n\n", task4_output)

Автор с самой высокой средней оценкой книг: 

                                                author  avg_rate
0                          J.K. Rowling/Mary GrandPré  4.283844
1                   Markus Zusak/Cao Xuân Việt Khương  4.264151
2                                      J.R.R. Tolkien  4.258446
3                                   Louisa May Alcott  4.192308
4                                        Rick Riordan  4.080645
5                                     William Golding  3.901408
6                                       J.D. Salinger  3.825581
7            Paulo Coelho/Alan R. Clarke/Özdemir İnce  3.789474
8   William Shakespeare/Paul Werstine/Barbara A. M...  3.787879
9                                           Dan Brown  3.754540
10                                         Lois Lowry  3.750000
11            George Orwell/Boris Grabnar/Peter Škerl  3.729730
12                                    Stephenie Meyer  3.662500
13                                     John Steinbeck  3.

<b>Выводы:</b>
* Самый высокий рейтинг у J.K. Rowling/Mary GrandPré - 4.28 баллов, её читают очень много людей. Как было выяснено во втором задании, она действительно в топе
* Австралийский писатель Markus Zusak занял второе место - 4.26 балла и следом за ним старина Толкиен - 4.25 балла

### Количество обзоров от пользователей, которые поставили больше 50 оценок

In [29]:
#task5 = """
#SELECT
#    AVG(SUBQ.rate_cnt) AS rate_cnt,
#    AVG(SUBQ1.rev_cnt) AS rev_cnt
#FROM
#    (SELECT
#        username,
#        COUNT(rating_id) AS rate_cnt
#    FROM
#        ratings
#    GROUP BY
#        username
#    HAVING
#        COUNT(rating_id) > 50) AS SUBQ
#RIGHT JOIN 
#    (SELECT
#        username,
#        COUNT(review_id) AS rev_cnt
#     FROM
#         reviews
#     GROUP BY
#         username) AS SUBQ1 ON SUBQ1.username = SUBQ.username;
#"""

In [52]:
task5 = """
SELECT
    AVG(text_cnt) AS rev_avg
FROM
    (SELECT
        username,
        COUNT(text) AS text_cnt
    FROM
        reviews
    WHERE
        username IN 
                (SELECT
                    username
                 FROM
                    ratings
                 GROUP BY
                    username
                 HAVING
                    COUNT(rating) > 50)
GROUP BY username) AS SUBQ;
"""

In [53]:
task5_output = sql_send(task5)

In [54]:
print("Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок: \n\n", 
      task5_output)

Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок: 

      rev_avg
0  24.333333


<b>Выводы:</b>
* Среднее количество оценкок книг 54, со средним количеством обзоров - 17

In [36]:
pd.read_sql("""
SELECT avg(count) 
FROM (SELECT username, count(text) 
        FROM reviews re WHERE re.username in (
                    SELECT r.username FROM ratings r 
                    GROUP BY r.username
                    HAVING count(r.rating) > 50)
GROUP BY username) as counts
""", con = engine)

,avg
0,24.333333


## Общие выводы

* Вначале я исследовал данные на аномалии и проанализировал общую информацию

Далее выполнил ряд задач и вот их выводы:

* После 2000 вышло 821 книг
* Очень популярны фентези книги британской писательницы - Джоан Роулинг, со средним рейтингом 4.28-4.41 и количеством обзоров - 6
* Лидер, среди издательств - британское издательство Penguin Books, которое выпустило 42 крупные книги
* Среди книг, количество оценок которых выше 50, книги Джоан Роулинг снова в топе - 4.41 и даже обогнала великого Толкиена
* В среднем 54 балла оставили пользователи, у которых насчитывается свыше 17 оценок книг